In [2]:
import heartpy as hp
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.signal import cheby2, sosfilt
from datetime import datetime, timedelta
from IPython.display import display

In [6]:
os.chdir('C:\\Users\\Owner\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Empatica')
os.getcwd()

def concatenate_data(SubjID):
    dirs = os.listdir('.\\' + SubjID)
    
    #remove baseline or zipped file if they exist
    if (SubjID + '_baseline') in dirs:
        dirs.remove(SubjID + '_baseline')
    if (SubjID + '_zipped') in dirs:
        dirs.remove(SubjID + '_zipped')
        
    df = pd.DataFrame()  
    for folder in dirs:
        wd = os.getcwd() + '\\' + SubjID + '\\' + folder
        eda = hp.get_data(wd + '\\EDA.csv')
        bvp = hp.get_data(wd + '\\BVP.csv')
        acc = hp.get_data(wd + '\\ACC.csv')
        temp = hp.get_data(wd + '\\TEMP.csv')
        
        init_time = datetime.fromtimestamp(eda[0]) #they all have the same initial time
        
        #removing timestamp and sampling frequency rows
        eda = eda[2:]
        bvp = bvp[2:]
        acc = acc[2:]
        temp = temp[2:]
        
        eda_interval = timedelta(seconds=1/4)
        bvp_interval = timedelta(seconds=1/64)
        acc_interval = timedelta(seconds=1/32)
        temp_interval = timedelta(seconds=1/4)
        
        eda_timestamps = [init_time + i * eda_interval for i in range(len(eda))]
        bvp_timestamps = [init_time + i * bvp_interval for i in range(len(bvp))]
        acc_timestamps = [init_time + i * acc_interval for i in range(len(acc))]
        temp_timestamps = [init_time + i * temp_interval for i in range(len(temp))]
        
        # Create a new DataFrame with the timestamps and the original data columns
        eda_df = pd.DataFrame(data = eda, columns=['EDA (4 Hz)'])
        eda_df['timestamp'] = eda_timestamps
        
        bvp_df = pd.DataFrame(data = bvp, columns=['BVP (64 Hz)'])
        bvp_df['timestamp'] = bvp_timestamps
        
        acc_df = pd.DataFrame(data = acc, columns=['ACC X (32 Hz)', 'ACC Y (32 Hz)', 'ACC Z (32 Hz)'])
        acc_df['timestamp'] = acc_timestamps
        
        temp_df = pd.DataFrame(data = temp, columns=['TEMP (4 Hz)'])
        temp_df['timestamp'] = temp_timestamps
        
        merged_df = pd.merge(bvp_df, eda_df, on='timestamp', how='left').merge(acc_df, on='timestamp', how='left').merge(temp_df, on='timestamp', how='left')
        
        df = pd.concat([df, merged_df])
    
    new_cols = ['timestamp','BVP (64 Hz)', 'EDA (4 Hz)','TEMP (4 Hz)','ACC X (32 Hz)','ACC Y (32 Hz)','ACC Z (32 Hz)',]
    df = df[new_cols]
    df = df.ffill(axis=0)
    
    return df
    


In [7]:
cov4 = concatenate_data('Cov4')

In [8]:
display(cov4)

,timestamp,BVP (64 Hz),EDA (4 Hz),TEMP (4 Hz),ACC X (32 Hz),ACC Y (32 Hz),ACC Z (32 Hz)
0,2023-02-28 10:03:33.000000,-0.00,0.000000,19.37,60.0,22.0,-2.0
1,2023-02-28 10:03:33.015625,-0.00,0.000000,19.37,60.0,22.0,-2.0
2,2023-02-28 10:03:33.031250,-0.00,0.000000,19.37,59.0,20.0,-5.0
3,2023-02-28 10:03:33.046875,-0.00,0.000000,19.37,59.0,20.0,-5.0
4,2023-02-28 10:03:33.062500,-0.00,0.000000,19.37,61.0,18.0,-13.0
...,...,...,...,...,...,...,...
3617719,2023-03-08 11:58:44.859375,0.49,0.032035,25.29,-60.0,-2.0,9.0
3617720,2023-03-08 11:58:44.875000,1.86,0.032035,25.29,-60.0,-2.0,9.0
3617721,2023-03-08 11:58:44.890625,3.27,0.032035,25.29,-60.0,-2.0,9.0
3617722,2023-03-08 11:58:44.906250,4.50,0.032035,25.29,-60.0,-2.0,9.0


In [9]:
cov8 = concatenate_data('Cov8')

In [10]:
display(cov8)

,timestamp,BVP (64 Hz),EDA (4 Hz),TEMP (4 Hz),ACC X (32 Hz),ACC Y (32 Hz),ACC Z (32 Hz)
0,2023-03-17 15:14:24.000000,-0.00,0.000000,34.23,13.0,-10.0,127.0
1,2023-03-17 15:14:24.015625,-0.00,0.000000,34.23,13.0,-10.0,127.0
2,2023-03-17 15:14:24.031250,-0.00,0.000000,34.23,7.0,-6.0,127.0
3,2023-03-17 15:14:24.046875,-0.00,0.000000,34.23,7.0,-6.0,127.0
4,2023-03-17 15:14:24.062500,-0.00,0.000000,34.23,14.0,-6.0,127.0
...,...,...,...,...,...,...,...
7964446,2023-03-31 22:47:01.468750,6.21,0.028186,28.29,6.0,-61.0,20.0
7964447,2023-03-31 22:47:01.484375,5.07,0.028186,28.29,6.0,-61.0,20.0
7964448,2023-03-31 22:47:01.500000,2.85,0.028186,28.29,6.0,-61.0,20.0
7964449,2023-03-31 22:47:01.515625,0.07,0.028186,28.29,6.0,-61.0,20.0


In [11]:
cov14 = concatenate_data('Cov14')

In [12]:
display(cov14)

,timestamp,BVP (64 Hz),EDA (4 Hz),TEMP (4 Hz),ACC X (32 Hz),ACC Y (32 Hz),ACC Z (32 Hz)
0,2023-04-15 07:19:19.000000,-0.00,0.0,382.21,-49.0,-26.0,-30.0
1,2023-04-15 07:19:19.015625,-0.00,0.0,382.21,-49.0,-26.0,-30.0
2,2023-04-15 07:19:19.031250,-0.00,0.0,382.21,-50.0,-26.0,-30.0
3,2023-04-15 07:19:19.046875,-0.00,0.0,382.21,-50.0,-26.0,-30.0
4,2023-04-15 07:19:19.062500,-0.00,0.0,382.21,-50.0,-26.0,-31.0
...,...,...,...,...,...,...,...
1560257,2023-04-28 20:02:36.015625,135.16,0.0,16.70,-15.0,-11.0,61.0
1560258,2023-04-28 20:02:36.031250,113.66,0.0,16.70,-17.0,-16.0,57.0
1560259,2023-04-28 20:02:36.046875,93.38,0.0,16.70,-17.0,-16.0,57.0
1560260,2023-04-28 20:02:36.062500,74.47,0.0,16.70,-16.0,-14.0,52.0


In [13]:
nwd = 'C:\\Users\\Owner\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\'

cov4.to_csv(nwd + 'Cov4_remote.csv', index=False)
cov8.to_csv(nwd + 'Cov8_remote.csv', index=False)
cov14.to_csv(nwd + 'Cov14_remote.csv', index=False)